In [122]:
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

market = [ "AD", "AR", "AT", "AU", "BE", "BG", "BO", "BR", "CA", "CH", "CL", "CO", "CR", "CY", 
      "CZ", "DE", "DK", "DO", "EC", "EE", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HU", 
      "ID", "IE", "IS", "IT", "JP", "LI", "LT", "LU", "LV", "MC", "MT", "MX", "MY", "NI", "NL", 
      "NO", "NZ", "PA", "PE", "PH", "PL", "PT", "PY", "SE", "SG", "SK", "SV", "TH", "TR", "TW", 
      "US", "UY", "VN" ]

CLIENT_ID = "63378b246c60499ea99838b8649cd63e"
CLIENT_SECRET = "684248b523bf4aabb7d8af129fd00f95"

SPOTIFY_USER = '6oup6y9z3si0zmr49hq1kze94?si=nWxGgzsVQWeHlR39UmT34Q'
scope = 'user-library-read'
token = util.prompt_for_user_token(SPOTIFY_USER, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri="http://google.com/")



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=63378b246c60499ea99838b8649cd63e&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQCYTzF_eTMapcR7OQ7uYGO7O9NLzvjQtnqsTWhvhvPtTuPIGE-Vd13fUXl1nCnSOyBHDEQT3IgZDPN9ZZwv0wZH2FNbgLiKhw402d6gJzLoVoxQFHb7pN7DtFFc0l4OgJxlkCgg2Ka2pSxgHS2ibKOkL_ty0QLqJcADjjiciFbNgWELtueTbPe8ZjvviqY7oDIv_0unM_zN82uEOg




## Get Saved Tracks

In [123]:
sp = spotipy.Spotify(auth=token)

In [20]:
user = sp.current_user()

In [21]:
user

{'display_name': 'Nihw',
 'external_urls': {'spotify': 'https://open.spotify.com/user/6oup6y9z3si0zmr49hq1kze94'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/6oup6y9z3si0zmr49hq1kze94',
 'id': '6oup6y9z3si0zmr49hq1kze94',
 'images': [{'height': None,
   'url': 'https://profile-images.scdn.co/images/userprofile/default/eafa171b314ceef47dea230e8d5984f0337bb9c1',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:6oup6y9z3si0zmr49hq1kze94'}

In [23]:
name = user['display_name']
follower = user['followers']['total']

In [29]:
import pandas as pd

In [30]:
df_saved_tracks = pd.DataFrame()
track_list = ''
added_ts_list = []
artist_list = []
title_list = []
more_songs = True
offset_index = 0
while more_songs:
    songs = sp.current_user_saved_tracks(offset=offset_index)
    for song in songs['items']:
        #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
        #get the time when the song was added
        added_ts_list.append(song['added_at'])
        #get the title of the song
        title_list.append(song['track']['name'])
        #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
    #get the track features and append into a dataframe
    track_features = sp.audio_features(track_list[:-1])
    df_temp = pd.DataFrame(track_features)
    df_saved_tracks = df_saved_tracks.append(df_temp)
    track_list = ''
    if songs['next'] == None:
        # no more songs in playlist
        more_songs = False
    else:
        # get the next n songs
        offset_index += songs['limit']
#include timestamp added, title and artists of a song
df_saved_tracks['added_at'] = added_ts_list
df_saved_tracks['song_title'] = title_list
df_saved_tracks['artists'] = artist_list

In [146]:
Popularity = []

for track in df_saved_tracks['uri']:
    pop = sp.track(track)
    Popularity.append(pop['popularity'])

In [147]:
df2 = pd.DataFrame({'popularity': Popularity})

In [166]:
df_saved_tracks.reset_index(inplace = True)

In [176]:
df_saved_tracks.drop('index', axis = 1, inplace = True)

In [184]:
frames = [df_saved_tracks,df2]
df = pd.concat(frames, axis =1, sort = False)

In [186]:
df.to_csv("Saved Tracks.csv")

## Get Top Read 

In [40]:
scope = 'user-top-read'
token2 = util.prompt_for_user_token(SPOTIFY_USER, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri="http://google.com/")



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=63378b246c60499ea99838b8649cd63e&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F&scope=user-top-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQA1ww0VAXInBCSfvQSZfFO2rUNbsaGsPP8N8kS76EaVEZKG5Glt5OO2Zfur0j3ACaMamNdl9loRNZGgTFX7tAq6QeADt_9_eF2_wBB0DC03SoHaQXXiNqVZnCNiCOO0Vx-xOSblA5SXAliThIp_Aqn7vbEWB2UM5Tf1_f6AfgGsvIv2xLXSU5bR91Q-t4cvZlmYFk-r8-CX




In [41]:
sp2 = spotipy.Spotify(auth=token2)

In [66]:
artists = sp2.current_user_top_artists()

In [93]:
top_artists = {}

top_artists['Name'] = []
top_artists['Genres'] = []
top_artists['Followers'] = []
top_artists['Popularity'] = []

for n in range(len(artists['items'])):
    top_artists['Name'].append(artists['items'][n]['name'])
    top_artists['Genres'].append(artists['items'][n]['genres'])
    top_artists['Followers'].append(artists['items'][n]['followers']['total'])
    top_artists['Popularity'].append(artists['items'][n]['popularity'])

In [96]:
top_artists = pd.DataFrame.from_dict(top_artists)
top_artists

,Name,Genres,Followers,Popularity
0,Martin Garrix,"[big room, edm, pop, progressive house, tropic...",10835067,88
1,Marshmello,"[brostep, progressive electro house]",14391537,91
2,R3HAB,"[big room, dance pop, dutch house, edm, electr...",1714710,84
3,Tiësto,"[big room, brostep, dance pop, edm, electro ho...",4493008,84
4,KSHMR,"[big room, deep big room, edm, electro house, ...",1038238,72
5,Alan Walker,[electro house],11776120,87
6,Jay Chou,"[c-pop, mandopop, taiwan pop]",1031080,73
7,Cash Cash,"[big room, dance pop, edm, electro house, elec...",454211,73
8,Avicii,"[big room, edm, pop]",14258628,90
9,Dimitri Vegas & Like Mike,"[belgian dance, belgian edm, big room, deep bi...",2497780,79


In [111]:
top_tracks = sp2.current_user_top_tracks(limit = 100)

In [114]:
top_songs = {}

top_songs['Song'] = []
top_songs['Artist'] = []
top_songs['url'] = []

for n in range(len(top_tracks['items'])):
    top_songs['Song'].append(top_tracks['items'][n]['name'])
    top_songs['Artist'].append(top_tracks['items'][n]['artists'][0]['name'])
    top_songs['url'].append(top_tracks['items'][n]['uri'])


In [115]:
top_songs = pd.DataFrame.from_dict(top_songs)
top_songs

,Song,Artist,url
0,May It Be,Enya,spotify:track:7LAJWSKK8JMIZAcblgUMS6
1,No Sleep (feat. Bonn),Martin Garrix,spotify:track:1ahVFh0ViDZr8LvkEVlq3B
2,Happier,Marshmello,spotify:track:2dpaYNEQHiRxtZbfNsse99
3,Here With Me,Marshmello,spotify:track:5icOoE6VgqFKohjWWNp0Ac
4,Bazaar - Official Sunburn Goa 2015 Anthem,KSHMR,spotify:track:712Hktc9DnU5u9uOSn1q1Y
5,Halfway There (feat. Lena Leon),Tiësto,spotify:track:3EKKIR8be1kfAAwaz1fzyS
6,Another Life - DubVision Remix,Afrojack,spotify:track:7iefpa6pT7TSARrHKGe3VJ
7,一路向北,Jay Chou,spotify:track:31RIphTQ9y4zStkH5xRWgW
8,High On Life (feat. Bonn),Martin Garrix,spotify:track:4ut5G4rgB1ClpMTMfjoIuy
9,One Thing Right - feat. Kane Brown,Marshmello,spotify:track:4hPpVbbakQNv8YTHYaOJP4
